# Reading Progress Module - Refine

This notebook demonstrates the utility of the OEA_py class notebook, and speeding up the process of refining/pseudonymizing the Reading Progress data. 

The steps outlined below describe how this notebook is used to refine tables originally from the Microsoft Education Insights data source:

- Set the workspace for where the tables are located. 
- 1 function is defined and used:
   1. **refine_reading_prog_corrected**: similar to the ```oea.refine()``` function, except reads from ```stage2/Ingested_Corrected``` rather than ```stage2/Ingested```.
      * *Important Note*: This function currently does not accomodate processing for change data over time; it is expected to be updated for production purposes.
   2. **refine_reading_progress_dataset**: 
      * transforms/aggregates data from Insights tables into two tables, relevant to the Reading Progress module.
      * uses the Reading_Progress module metadata.csv to pseudonymize each table according to whether it is to be hashed, masked, or has no-operation.

In [ ]:
workspace = 'dev'

In [ ]:
%run OEA_py

In [ ]:
# 1) set the workspace (this determines where in the data lake you'll be writing to and reading from).
# You can work in 'dev', 'prod', or a sandbox with any name you choose.
# For example, Sam the developer can create a 'sam' workspace and expect to find his datasets in the data lake under oea/sandboxes/sam
oea.set_workspace(workspace)

In [ ]:
# 2) this step refines the data through the use of metadata (this is where the pseudonymization of the data occurs).
def refine_reading_prog_corrected(df, table_name, metadata=None, primary_key='id'):
    source_path = f'stage2/Ingested_Corrected/reading_progress/v0.1/activity'
    sink_general_path = f'stage2/Refined/reading_progress/v0.1/general/{table_name}'
    sink_sensitive_path = f'stage2/Refined/reading_progress/v0.1/sensitive/{table_name}_lookup'

    # NOTE: Currently does not accomodate change data; this is expected to be updated for production purposes
    #df_changes = oea.get_latest_changes(source_path, sink_general_path)
    spark_schema = oea.to_spark_schema(metadata)
    df = oea.modify_schema(df, spark_schema)        

    if df.count() > 0:
        df_pseudo, df_lookup = oea.pseudonymize(df, metadata)
        oea.upsert(df_pseudo, sink_general_path, primary_key)
        oea.upsert(df_lookup, sink_sensitive_path, primary_key)
        oea.add_to_lake_db(sink_general_path)
        oea.add_to_lake_db(sink_sensitive_path)
        logger.info(f'Processed {df.count()} rows from {source_path} into stage2/Refined')
    else:
        logger.info(f'No updated rows in {source_path} to process.')
    return df.count()

def refine_reading_progress_dataset(tables_source):
    # read in relevant tables for data transformation
    base_path = tables_source
    df_activity = oea.load(base_path + '/activity')
    df_aaduserpersonmapping = oea.load(base_path + '/AadUserPersonMapping')
    df_person = oea.load(base_path + '/Person/')
    df_personOrgRole = oea.load(base_path + '/PersonOrganizationRole')
    df_organization = oea.load(base_path + '/Organization')
    df_refDefinition = oea.load(base_path + '/RefDefinition')
    # separate student frame, subset and refine data
    dfStudent = df_personOrgRole.join(df_person, df_personOrgRole.PersonId == df_person.Id, how='inner')
    dfStudent = dfStudent.select('PersonId', 'Surname', 'GivenName', 'MiddleName', 'RefRoleId', 'RefGradeLevelId', 'OrganizationId')
    dfStudent = dfStudent.join(df_organization, dfStudent.OrganizationId == df_organization.Id, how='inner').withColumnRenamed('Name', 'OrganizationName')
    dfStudent = dfStudent.select('PersonId', 'Surname', 'GivenName', 'MiddleName', 'RefRoleId', 'RefGradeLevelId', 'OrganizationId', 'OrganizationName')
    dfStudent = dfStudent.join(df_refDefinition, dfStudent.RefRoleId == df_refDefinition.Id, how='inner').withColumnRenamed('Code', 'PersonRole')
    dfStudent = dfStudent.select('PersonId', 'Surname', 'GivenName', 'MiddleName', 'PersonRole', 'RefGradeLevelId', 'OrganizationId', 'OrganizationName')
    dfStudent = dfStudent.filter(dfStudent['PersonRole'] == 'Student')
    dfStudent = dfStudent.join(df_refDefinition, dfStudent.RefGradeLevelId == df_refDefinition.Id, how='left').withColumnRenamed('Code', 'StudentGrade')
    dfStudent = dfStudent.select('PersonId', 'Surname', 'GivenName', 'MiddleName', 'PersonRole', 'StudentGrade', 'OrganizationId', 'OrganizationName')
    df_aaduserpersonmapping = df_aaduserpersonmapping.withColumnRenamed('PersonId', 'id')
    dfStudent = dfStudent.join(df_aaduserpersonmapping, dfStudent.PersonId == df_aaduserpersonmapping.id, how='inner').withColumnRenamed('ObjectId', 'AadUserId')
    dfStudent = dfStudent.select('PersonId', 'AadUserId', 'Surname', 'GivenName', 'MiddleName', 'PersonRole', 'StudentGrade', 'OrganizationId', 'OrganizationName')

    refine_reading_prog_corrected(dfStudent, 'Student', metadata['Student'], 'PersonId_pseudonym')
    # refine reading progress data from Insights activity table
    dfReadingProgress = df_activity.where("AppName == 'ReadingProgress'")
    dfReadingProgress = dfReadingProgress.select('ActorId', 'SignalId', 'SignalType', 'StartTime', 'AppName', 'Action', 'ClassId', 'ReadingSubmissionWordsPerMinute', 'ReadingSubmissionAccuracyScore', \
                                        'ReadingSubmissionRepetitionsCount', 'ReadingSubmissionInsertionsCount', 'ReadingSubmissionMispronunciationCount', 'ReadingSubmissionObmissionCount', 'ReadingSubmissionAttemptNumber', \
                                        'ReadingAssignmentWordCount', 'ReadingAssignmentFleschKincaidGradeLevel', 'ReadingAssignmentLanguage')
    dfReadingProgress = dfReadingProgress.withColumnRenamed('ActorId', 'AadUserId').withColumnRenamed('ClassId', 'AadGroupId')
    dfReadingProgress = dfReadingProgress.withColumn('ReadingSubmissionAccuracyScore', dfReadingProgress['ReadingSubmissionAccuracyScore'].cast(DoubleType()))
    dfReadingProgress = dfReadingProgress.withColumn('ReadingSubmissionRepetitionsRate', F.col('ReadingSubmissionRepetitionsCount')/F.col('ReadingAssignmentWordCount') * 100) 
    dfReadingProgress = dfReadingProgress.withColumn('ReadingSubmissionRepetitionsRate', F.round(F.col('ReadingSubmissionRepetitionsRate'), 3))
    dfReadingProgress = dfReadingProgress.withColumn('ReadingSubmissionMispronunciationRate', F.col('ReadingSubmissionMispronunciationCount')/F.col('ReadingAssignmentWordCount') * 100) 
    dfReadingProgress = dfReadingProgress.withColumn('ReadingSubmissionMispronunciationRate', F.round(F.col('ReadingSubmissionMispronunciationRate'), 3))
    dfReadingProgress = dfReadingProgress.withColumn('ReadingSubmissionInsertionsRate', F.col('ReadingSubmissionInsertionsCount')/F.col('ReadingAssignmentWordCount') * 100) 
    dfReadingProgress = dfReadingProgress.withColumn('ReadingSubmissionInsertionsRate', F.round(F.col('ReadingSubmissionInsertionsRate'), 3))
    dfReadingProgress = dfReadingProgress.withColumn('ReadingSubmissionObmissionRate', F.col('ReadingSubmissionObmissionCount')/F.col('ReadingAssignmentWordCount') * 100) 
    dfReadingProgress = dfReadingProgress.withColumn('ReadingSubmissionObmissionRate', F.round(F.col('ReadingSubmissionObmissionRate'), 3))

    try:
        refine_reading_prog_corrected(dfReadingProgress, 'ReadingProgress_activity', metadata['ReadingProgress_activity'], 'SignalId')
    except AnalysisException as e:
        # This means the table may have not been properly refined due to errors with the primary key not aligning with columns expected in the lookup table.
        pass
    
    logger.info('Finished refining Reading Progress tables.')

In [ ]:
metadata = oea.get_metadata_from_url('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Reading_Progress/data/metadata.csv')
refine_reading_progress_dataset('stage2/Ingested_Corrected/reading_progress/v0.1')